## Import Packages

In [23]:
import os
import geopandas as gpd
import fiona

import hvplot.xarray
import hvplot.pandas
import cartopy.crs as ccrs
import holoviews as hv
import geoviews as gv

import panel as pn

In [24]:
# TODO: add widgets to let user drop in their onw files
cpes = gpd.read_file('input-data/CPES/cpes.shp')
paddocks = gpd.read_file('input-data/Paddock boundaries/paddock-boundaries.shp').to_crs(cpes.crs)

# set up map tiles for basemap
map_tiles = hv.element.tiles.EsriImagery().opts(width=600, height=400)

dst_crs = 'EPSG:3857'

cpes.crs

<Geographic 2D CRS: EPSG:7844>
Name: GDA2020
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: Australia including Lord Howe Island, Macquarie Island, Ashmore and Cartier Islands, Christmas Island, Cocos (Keeling) Islands, Norfolk Island. All onshore and offshore.
- bounds: (93.41, -60.55, 173.34, -8.47)
Datum: Geocentric Datum of Australia 2020
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [25]:
cpes = cpes.to_crs(dst_crs)
paddocks = paddocks.to_crs(dst_crs)

In [26]:
cpes_unify = cpes.dissolve()
centroid = cpes_unify.centroid

difference_areas = gpd.overlay(paddocks, cpes, how='symmetric_difference') 
outside_areas = gpd.overlay(difference_areas, cpes, how='difference')
inside_areas = paddocks.clip(cpes)

In [30]:
parcels_map = cpes_unify.hvplot(alpha=0.5, geo=True)
outside_map = outside_areas.hvplot(geo=True).opts(title='submitted areas that fall outside land parcels')
inside_map = inside_areas.hvplot(geo=True).opts(title='submitted areas that are inside land parcels')

In [31]:
pn.Row(
    pn.Column(
    (map_tiles * parcels_map * outside_map),
    (map_tiles * parcels_map * inside_map)
    )
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'74c28916-9059-4eba-8402-23a47e633dd5': {'version…

In [29]:
#export new areas as geojson
output_crs = '4326'


#If the CRS is set to something else, the geojson will still save but it won't actually be in 4326. Paddocks was set to 3857 and wrote out with pos/neg values.
inside_areas.to_file('output-data/modified-user-boundaries.geojson', driver='GeoJSON')
outside_areas.to_file('output-data/outside-parcel-user-boundaries.geojson', driver='GeoJSON')
paddocks.to_file('output-data/input-user-boundaries.geojson', driver='GeoJSON')